In [1]:
#!pip install requests

In [2]:
#!pip install requests openpyxl

In [3]:
#!pip install tqdm

In [4]:
import requests
import json  # already built-in
import pandas as pd
import time
import re
# Import and set up tqdm
from tqdm import tqdm
tqdm.pandas()  # enables progress bar with .progress_apply



# Function to extract ingredients using Mistral via Ollama
def extract_ingredients_from_description(description):
    if pd.isna(description) or description.strip() == "":
        return ""

    prompt = f"""Extract all ingredients and their quantities from the following cooking description.
Only output a clean bullet list (one item per line), like:
> 1 cup flour
> 2 tbsp sugar

Text:
\"\"\"{description}\"\"\"
"""

    try:
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": "mistral",
                "prompt": prompt,
                "stream": False
            },
            timeout=60  # seconds
        )

        if response.status_code == 200:
            result = response.json()
            return result['response'].strip()
        else:
            return f" Error: {response.status_code}"
    except Exception as e:
        return f" Exception: {str(e)}"

# Load your Excel file
input_path = 'edit.xlsx'
df = pd.read_excel(input_path)

# Optional: Show first few rows
df.head()


# Apply the ingredient extractor to each row

df['ingredients'] = df['description'].progress_apply(extract_ingredients_from_description)


# Optional: wait between rows to be nice to the model (for big files)

# Save the output to a new Excel file
output_path = 'output_with_ingredients.xlsx'
df.to_excel(output_path, index=False)

print(f" Done! Saved extracted ingredients to: {output_path}")


100%|██████████████████████████████████████████████████████████████████████████████| 429/429 [2:02:21<00:00, 17.11s/it]


✅ Done! Saved extracted ingredients to: output_with_ingredients.xlsx
